In [1]:
import numpy as np

In [2]:
%reload_ext autoreload
%autoreload 2

from MTPHandler import Plate

In [3]:
path = "/Users/max/Documents/enzymeml/alaric/EnzymeML_SLAC_5xConcentration_pH2.0-3.5/ABTS_EnzymeML_340nm_420nm_5x_35deg_pH3.5.txt"
time = np.linspace(0, 15, 31)
time_unit = "min"

# Load plate data
plate = Plate.from_file(path, time, time_unit)

# Define species
buffer = plate.add_reactant(id="s1", name="buffer", constant=True)
abts = plate.add_reactant(id="s0", name="ABTS", constant=False,
                          inchi="InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/b19-17-,20-18+")
slac = plate.add_protein(id="p0", name="SLAC",
                         constant=True, sequence="MSSKSKPKDVKV")

# Define well conditions for each species
# Define buffer concentration
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=buffer,
    init_concs=100,
    conc_unit="umol / l"
)

# Define substrate concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C", "D", "E", "F"],
    species=abts,
    init_concs=[0, 5, 10, 15, 25, 50, 75, 100, 150, 200],
    conc_unit="umol / l"
)

# Define enzyme concentrations
plate.assign_species_conditions_to_rows(
    row_ids=["A", "B", "C"],
    species=slac,
    init_concs=4,
    conc_unit="umol / l"
)

# Blank data
plate.blank_species(buffer, 340)
plate.blank_species(slac, 340)

### Create instance of ```Standard```

In [4]:
abts_calibrator = plate.calibrate(abts, 340)

s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False
s0 was blanked: False


In [5]:
linear_model = abts_calibrator.add_to_models(
    name="linear", equation="a * concentration = signal"
)
quadratic_model = abts_calibrator.add_to_models(
    name="quadratic", equation="a * concentration^2 + b * concentration = signal"
)
cubic_model = abts_calibrator.add_to_models(
    name="cubic", equation="a * concentration^3 + b * concentration^2 + c * concentration = signal"
)
rational_model = abts_calibrator.add_to_models(
    name="rational", equation="a * concentration / (b + concentration) = signal"
)

abts_calibrator.fit_models()

CalibrationModel
├── id = calibrationmodel1
├── name = linear
├── equation = a * concentration = signal
├── parameters
│   └── 0
│       └── Parameter
│           ├── id = parameter0
│           ├── name = a
│           ├── value = 0.018991787683955583
│           ├── init_value = 0.1
│           ├── standard_error = 0.0003576140010834523
│           ├── lower_bound = -inf
│           └── upper_bound = inf
├── was_fitted = True
├── calibration_range
│   └── CalibrationRange
│       ├── id = calibrationrange1
│       ├── conc_lower = 5.0
│       ├── conc_upper = 150.0
│       ├── signal_lower = 0.07969135974636946
│       └── signal_upper = 2.7944619534978234
└── statistics
    └── FitStatistics
        ├── id = fitstatistics1
        ├── aic = -98.32491050898993
        ├── bic = -97.14685667864198
        ├── r2 = 0.9815817838306456
        └── rmsd = 0.12367448298983127

CalibrationModel
├── id = calibrationmodel2
├── name = quadratic
├── equation = a * concentration^2 + b * concentr

,AIC,R squared,RMSD
Model Name,,,
cubic,-114,0.9919,0.0823
quadratic,-104,0.9864,0.1063
rational,-103,0.9860,0.1079
linear,-98,0.9816,0.1237


In [6]:
abts_calibrator.visualize(linear_model)

In [7]:
abts_standard = abts_calibrator.save_fitted_model(linear_model)
print(abts_standard)

Standard
├── id = standard1
├── species_id = s0
├── name = ABTS
├── wavelength = 340.0
├── samples
│   ├── 0
│   │   └── Sample
│   │       ├── id = D2
│   │       ├── concentration = 5.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.08641605681905236
│   ├── 1
│   │   └── Sample
│   │       ├── id = D3
│   │       ├── concentration = 10.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.18111455485537442
│   ├── 2
│   │   └── Sample
│   │       ├── id = D4
│   │       ├── concentration = 15.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.29443428891193385
│   ├── 3
│   │   └── Sample
│   │       ├── id = D5
│   │       ├── concentration = 25.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.5018776976691343
│   ├── 4
│   │   └── Sample
│   │       ├── id = D6
│   │       ├── concentration = 50.0
│   │       ├── conc_unit = umol / l
│   │       └── signal = 0.9929862158855982
│   ├── 5
│   │   └── Sample
│   │       ├──

In [23]:
enzymeml = plate.to_enzymeml(
    name="SLAC concentration assay data",
    reactant=abts,
    standard=abts_standard,
    protein=slac,
    wavelength=340,
)

In [24]:
print(enzymeml)

EnzymeMLDocument
├── id = enzymemldocument1
├── name = SLAC concentration assay data
├── created = 2023-09-12 10:38:34
├── vessels
│   └── 0
│       └── Vessel
│           ├── id = plate0
│           ├── name = MTP 96 well
│           ├── volume = 200.0
│           ├── unit = ul
│           └── constant = True
├── proteins
│   └── 0
│       └── Protein
│           ├── id = p0
│           ├── name = SLAC
│           ├── vessel_id = plate0
│           ├── constant = True
│           ├── sequence = MSSKSKPKDVKV
│           └── ontology = SBO:0000013
├── reactants
│   ├── 0
│   │   └── Reactant
│   │       ├── id = s1
│   │       ├── name = buffer
│   │       ├── vessel_id = plate0
│   │       ├── constant = True
│   │       └── ontology = SBO:0000247
│   └── 1
│       └── Reactant
│           ├── id = s0
│           ├── name = ABTS
│           ├── vessel_id = plate0
│           ├── constant = False
│           ├── inchi = InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(2